In [1]:
install.packages('PrevMap')
library(PrevMap)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: maxLik

Loading required package: miscTools


Please cite the 'maxLik' package as:
Henningsen, Arne and Toomet, Ott (2011). maxLik: A package for maximum likelihood estimation in R. Computational Statistics 26(3), 443-458. DOI 10.1007/s00180-010-0217-1.

If you have questions, suggestions, or comments regarding the 'maxLik' package, please use a forum or 'tracker' at maxLik's R-Forge site:
https://r-forge.r-project.org/projects/maxlik/

Loading required package: raster

Loading required package: sp


Attaching package: ‘raster’


The following object is masked from ‘package:maxLik’:

    maxValue


Loading required package: pdist

Loading required package: Matrix



In [2]:
install.packages('geoR')
library(geoR)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Warning message:
“no DISPLAY variable so Tk is not available”
--------------------------------------------------------------
 Analysis of Geostatistical Data
 For an Introduction to geoR go to http://www.leg.ufpr.br/geoR
 geoR version 1.8-1 (built on 2020-02-08) is now loaded
--------------------------------------------------------------




In [3]:
data = read.csv("data_bayes2.csv")
dim(data)
summary(data)

[1] 15196     9

       X               nr               id              frag       
 Min.   :    1   Min.   :222157   Min.   :314301   Min.   :0.0000  
 1st Qu.: 3800   1st Qu.:225956   1st Qu.:359419   1st Qu.:0.0000  
 Median : 7598   Median :229754   Median :384838   Median :0.0000  
 Mean   : 7598   Mean   :229754   Mean   :384330   Mean   :0.1714  
 3rd Qu.:11397   3rd Qu.:233553   3rd Qu.:410083   3rd Qu.:0.0000  
 Max.   :15196   Max.   :237352   Max.   :447230   Max.   :1.0000  
     COMUNE          x.pxl             y.pxl             x.mun        
 Belluno:15196   Min.   :4484879   Min.   :2549742   Min.   :4492645  
                 1st Qu.:4489879   1st Qu.:2557642   1st Qu.:4492645  
                 Median :4492679   Median :2561042   Median :4492645  
                 Mean   :4492626   Mean   :2561155   Mean   :4492645  
                 3rd Qu.:4495479   3rd Qu.:2565042   3rd Qu.:4492645  
                 Max.   :4499579   Max.   :2571442   Max.   :4492645  
     y.mun        
 Min.   

In [13]:
data = distinct(data,id, .keep_all= TRUE)
dim(data)

[1] 54864     9

In [4]:
head(data)

,X,nr,id,frag,COMUNE,x.pxl,y.pxl,x.mun,y.mun
,<int>,<int>,<int>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,222157,314301,1,Belluno,4484879,2560542,4492645,2561178
2,2,222158,314302,1,Belluno,4484879,2560442,4492645,2561178
3,3,222159,315204,1,Belluno,4484979,2560642,4492645,2561178
4,4,222160,315205,1,Belluno,4484979,2560542,4492645,2561178
5,5,222161,315206,0,Belluno,4484979,2560442,4492645,2561178
6,6,222162,315207,0,Belluno,4484979,2560342,4492645,2561178


In [5]:
data = data[,-c(1,3)]
head(data)

,nr,frag,COMUNE,x.pxl,y.pxl,x.mun,y.mun
,<int>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
1,222157,1,Belluno,4484879,2560542,4492645,2561178
2,222158,1,Belluno,4484879,2560442,4492645,2561178
3,222159,1,Belluno,4484979,2560642,4492645,2561178
4,222160,1,Belluno,4484979,2560542,4492645,2561178
5,222161,0,Belluno,4484979,2560442,4492645,2561178
6,222162,0,Belluno,4484979,2560342,4492645,2561178


In [ ]:
n = nrow(data)
n.mun = length(unique(data$COMUNE))

ID.coords = create.ID.coords(data,coords=~x.mun+y.mun)

control.mcmc = control.mcmc.Bayes(n.sim=10,
                                   burnin=0,
                                   thin=1,
                                   h.theta1=0.05,
                                   h.theta2=0.05,
                                   L.S.lim=c(1,50),
                                   epsilon.S.lim=c(0.01,0.02),
                                   start.beta=0,
                                   start.sigma2=1,
                                   start.phi=0.15,
                                   start.S=rep(0,n),
                                  binary = TRUE)

cp = control.prior(beta.mean=0,
                    beta.covar=1,
                    log.normal.phi=c(log(0.15),0.05),
                    log.normal.sigma2=c(log(1),0.1))
gc()

fit.Bayes = binary.probit.Bayes(formula=frag~1,
                                 coords=~x.pxl+y.pxl,
                                 data=data,
                                 ID.coords = ID.coords,
                                 control.prior=cp,
                                 control.mcmc=control.mcmc,
                                 kappa=2)

Warning message in control.mcmc.Bayes(n.sim = 10, burnin = 0, thin = 1, h.theta1 = 0.05, :
“inclusion of the nugget effect is not available for the binary model”


,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2613767,139.6,4935628,263.6,3607818,192.7
Vcells,4230817,32.3,10146329,77.5,7902897,60.3
